In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tqdm
from tqdm.auto import tqdm as tqdmp
tqdmp.pandas()

# Work with phash
import imagehash

import cv2, os
import skimage.io as io
from PIL import Image

# ignoring warnings
import warnings

warnings.simplefilter("ignore")
import torch
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tqdm
from tqdm.auto import tqdm as tqdmp
tqdmp.pandas()

import cv2, os
import skimage.io as io
from PIL import Image

# ignoring warnings
import warnings

warnings.simplefilter("ignore")
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch.nn.functional as F

import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

class DataLoader:
    def __init__(self, path, outputpath):
        self.path = path
        self.outputpath = outputpath

    def load_df(self, csvpath, imagepath):
        self.csvpath = ('/').join([self.path, csvpath])
        self.df = pd.read_csv(self.csvpath)
        return self.df

    def generate_image_path(self):
        self.train_df = self.load_df('train.csv', 'train_images')
        self.test_df = self.load_df('test.csv', 'test_images')

        print(f"train: {self.train_df.shape}  test: {self.test_df.shape}")
        print(f"unique labels: {self.train_df.label_group.nunique()}")

        self.preprocess_train(self.train_df)
        self.preprocess_test(self.test_df)

    def preprocess_train(self, df):
        df['path'] = self.path + '/train_images/' + df['image']
        df.to_csv(self.outputpath + '/train_proc.csv')

    def preprocess_test(self, df):
        df['path'] = self.path + '/test_images/' + df['image']
        df.to_csv(self.outputpath + '/test_proc.csv')


class ImageHandler:
    def image_shape(self, image_path):
        im = cv2.imread(image_path)
        return str(im.shape)

    def standardize_image(self, width, height, orig_path, new_path):
        # Generate shape of image, resize them to 200 x 200, as that is the min found in this set
        # Then generate the array which represents each image
        # Divide them all by 255 to scale them
        # Persist these images in disk before next steps

        torch.manual_seed(17)
        self.image_viz(orig_path)
        image = cv2.imread(orig_path)
        dim = (width, height)
        # resize image
        resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
        tran = transforms.ToTensor()  # Convert the numpy array (C, H, W) Tensor format and /255 normalize to [0, 1.0]
        img_tensor = tran(resized) # (C,H,W), channel order (B, G, R)
        torch.save(img_tensor, new_path)

    def image_viz(self, image_path):
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.axis('off')

    def plot_image_pairs(self, df, index1, index2):
        for idx, path in enumerate([df.loc[index1, 'path'], df.loc[index2, 'path']]):
            plt.subplot(1, 2, idx + 1)
            self.image_viz(path)
        plt.show()

    def plot_image(self, df, index):
        for idx, path in enumerate([df.loc[index, 'path']]):
            plt.subplot(1, 2, idx + 1)
            self.image_viz(path)
        plt.show()



class MatchFinder:
    def match_matrix(self, phash_array):
        """ A function that checks for matches by phash value,
        takes phash values as input, outputs diff matrix as a df """
        phashs = phash_array.apply(lambda x: imagehash.hex_to_hash(x))
        phash_matrix = pd.DataFrame()
        for idx, i in enumerate(phash_array):
            phash_matrix = pd.concat([phash_matrix, phashs - imagehash.hex_to_hash(i)], axis=1)
        phash_matrix.columns = range(len(phash_array))
        return phash_matrix


class GenerateDataset:

    def generate_matching_pairs(self, path, outputpath, matches):
        matchingsets = []
        for i in range(len(matches)):
            matchingsets.append(matches.iloc[i, :][matches.iloc[i, :] == 0].index.values)
        matchingsets = pd.Series(matchingsets)
        pairs = matchingsets[matchingsets.apply(lambda x: len(x) == 2)]
        triplets = matchingsets[matchingsets.apply(lambda x: len(x) == 3)]
        quartets = matchingsets[matchingsets.apply(lambda x: len(x) == 4)]

        matching_pairs = []
        for idx, val in pairs.items():
            matching_pairs.append(val)

        for idx, value in triplets.items():
            matching_pairs.append([value[0], value[1]])
            matching_pairs.append([value[1], value[2]])
            matching_pairs.append([value[0], value[2]])

        for idx, value in quartets.items():
            matching_pairs.append([value[0], value[1]])
            matching_pairs.append([value[1], value[2]])
            matching_pairs.append([value[0], value[2]])
            matching_pairs.append([value[0], value[3]])
            matching_pairs.append([value[1], value[3]])
            matching_pairs.append([value[2], value[3]])

        final = list(set([tuple(t) for t in matching_pairs]))

        matching = pd.DataFrame().from_records(final, columns=['one', 'two'])
        matching = matching.sort_values(by=['one'])
        matching.to_csv(outputpath + '/matching_pairs.csv')

        non_matching_pairs = []
        counter = 0
        for j in range(0, len(matching_pairs)):
            if j not in matching['one']:
                #doesn't work, manually edit if there is a subsequent image pair match
                #non_matching_pairs.append([j, j+ 1])
                counter += 1
            if counter == matching.shape[0]:
                break
        non_matching = pd.DataFrame().from_records(non_matching_pairs, columns=['one', 'two'])
        non_matching = non_matching.sort_values(by=['one'])
        non_matching.to_csv(outputpath + '/non_matching_pairs.csv')
        return

    def merged_dataset(self, index_df, train_df):
        # Merge data from training set with image pairs [posting_id, image, image_phash, title, label_group]
        # from train_df
        train_df['index'] = train_df.index

        dataset_one = train_df.loc[index_df['one']]
        dataset_one['idx'] = index_df.index
        dataset_two = train_df.loc[index_df['two']]
        dataset_two['idx'] = index_df.index

        dataset = pd.merge(dataset_one, dataset_two, on='idx', suffixes=['_1', '_2'])
        dataset = pd.merge(dataset, index_df, left_index=True, right_index=True)
        return dataset

In [ ]:
def initial_processing(path, outputpath):
    dataLoader = DataLoader(path, outputpath)
    imageHandler = ImageHandler()

    dataLoader.generate_image_path()

    train_df = pd.read_csv(outputpath + '/train_proc.csv', 'train_images')
    test_df = pd.read_csv(outputpath + '/test_proc.csv', 'test_images')
#     print(train_df.head)
#     imageHandler.plot_image_pairs(train_df, 11, 12)
#     imageHandler.plot_image(train_df, 11)

    # Since the process of composing a matrix is quite resource-intensive,
    # for clarity, we will take only the first thousand images.
    # TODO:
    # A good strategy to expand on this matching dataset
    # is to look for same labelgroup and generate matching pairs within each labelgroup

    train_1000 = train_df.iloc[:1000, :]
#     train_1000 = train_1000.drop(columns=['Unnamed: 0'])
    matchFinder = MatchFinder()
    matches = matchFinder.match_matrix(train_1000['image_phash'])

    generator = GenerateDataset()
    generator.generate_matching_pairs(path, matches)

    # We have a dataset generated with first 1000 entries from the train dataset
    df1 = pd.read_csv(outputpath + '/matching_pairs.csv')
    df1['label'] = True
    df2 = pd.read_csv(outputpath + '/non_matching_df.csv')
    df1 = df1.drop(columns=['Unnamed: 0'])
    df2 = df2.drop(columns=['Unnamed: 0'])
#     df2['label'] = False
    merged = pd.concat([df1, df2])
    merged = merged.reset_index(drop=True)
    merged.to_csv(outputpath + '/merged.csv')

    # start with 'merged.csv' in path
    index_df = pd.read_csv(outputpath + '/merged.csv')
    train_df = pd.read_csv(outputpath + '/train_proc.csv')
    train_df = train_df.drop(columns=['Unnamed: 0'])
    index_df = index_df.drop(columns=['Unnamed: 0'])
    train_1000 = train_df.iloc[:1000, :]
    generator = GenerateDataset()
    merged_df = generator.merged_dataset(index_df, train_1000)
    pd.set_option('display.max_columns', None)
    merged_df = merged_df.reset_index(drop=True)
    merged_df.to_csv(outputpath + '/merged_with_data.csv')

In [ ]:
path = '/kaggle/input/shopee-product-matching'
# train_df = initial_processing(path, '/kaggle/working')

In [ ]:
path = '/kaggle/input/shopee-product-matching'
output = '/kaggle/working'
train_df = pd.read_csv(path + '/train.csv')
similar_products = train_df.groupby(['label_group'])['image'].apply(list)

In [ ]:
# Build a data set with similar product images
# make equal pairs of similar and dissimilar image pairs, labeled true and false for whether they are for similar products or not
def generate_matching_pairs(similar_products):
    pairs = []
    triplets = []
    quartets = []
    
    for x in similar_products:
        if(len(x) == 2):
            pairs.append(x)
        elif len(x) == 3:
            triplets.append(x)
        else:
            quartets.append(x)

    matching_pairs = []
    for val in pairs:
        matching_pairs.append(val)

    for value in triplets:
        matching_pairs.append([value[0], value[1]])
        matching_pairs.append([value[1], value[2]])
        matching_pairs.append([value[0], value[2]])

    for value in quartets:
        matching_pairs.append([value[0], value[1]])
        matching_pairs.append([value[1], value[2]])
        matching_pairs.append([value[0], value[2]])
        matching_pairs.append([value[0], value[3]])
        matching_pairs.append([value[1], value[3]])
        matching_pairs.append([value[2], value[3]])

    final = list(set([tuple(t) for t in matching_pairs]))
    matching = pd.DataFrame().from_records(final, columns=['image1', 'image2'])
    matching['label'] = 1
    return matching

In [ ]:
df = generate_matching_pairs(similar_products.tolist())

df.shape

In [ ]:
df.to_csv(output + '/matching_pairs_all.csv')

In [ ]:
def generate_non_matching_pairs(similar_products, size):
    non_matching_pairs = []
    for i in range(1, len(similar_products)):
        list1 = similar_products[i-1:i][0]
        list2 = similar_products[i:i+1][0]
        list3 = zip(list1, list2)
        for item in list3:
            non_matching_pairs.append(item)
        if len(non_matching_pairs) > size: 
            break
        
    final = list(set([tuple(t) for t in non_matching_pairs]))
    non_matching = pd.DataFrame().from_records(final, columns=['image1', 'image2'])
    non_matching['label'] = 0
    return non_matching

In [ ]:
non_df = generate_non_matching_pairs(similar_products.tolist(), 24000)
non_df.to_csv(output + '/non_matching_pairs_all.csv')

In [ ]:
non_df.shape

In [ ]:
dataset = pd.concat([df, non_df])
dataset.to_csv(output + '/dataset.csv')

In [ ]:
class ShopeeImagePairDataset(Dataset):

    def __init__(self, size, csv_file):
        df = pd.read_csv(csv_file)
        
        #24k samples for each class. Randomize inputs     
        similar = df.head(24000).sample(n=size,replace=False)
        dissimilar = df.tail(24000).sample(n=size,replace=False)
        self.dataframe = pd.concat([similar, dissimilar], ignore_index=True)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img1_path = '../input/shopee-product-matching/train_images/' + self.dataframe['image1'][idx]
        img2_path = '../input/shopee-product-matching/train_images/' + self.dataframe['image2'][idx]
        
        image1 = self.transform(img1_path)
        image2 = self.transform(img2_path)
        
        label = torch.tensor(self.dataframe['label'][idx])
        return image1, image2, label.int()
    
    def transform(self, image_path):
        image = io.imread(image_path)
        dim = (200, 200)
        resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
        tran = transforms.ToTensor()  # Convert the numpy array (C, H, W) Tensor format and /255 normalize to [0, 1.0]
        img_tensor = tran(resized) # (C,H,W), channel order (B, G, R)
        return img_tensor

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
image_path = output + '/dataset.csv'


In [ ]:
def create_data_sets(size, batch_size):
    size = size
    batch_size = batch_size
    validation_split = .2
    shuffle_dataset = True
    random_seed= 42

    # Creating data indices for training and validation splits:
    shopee_dataset = ShopeeImagePairDataset(size, image_path)
    dataset_size = len(shopee_dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(shopee_dataset, batch_size=batch_size, 
                                               sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(shopee_dataset, batch_size=batch_size,
                                                    sampler=valid_sampler)
#     print(len(train_loader))
#     print(len(validation_loader))
    return train_loader, validation_loader

In [ ]:
# training and validation loss were calculated after every epoch
def train(model, train_loader, val_loader, num_epochs):
    train_losses = []
    val_losses = []
    accuracy = []
    cur_step = 0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        print("Starting epoch " + str(epoch+1))
        for i, data in enumerate(train_loader,0):
            img0, img1, label = data
            img0, img1, label = img0.to(device), img1.to(device) , label.to(device)
            optimizer.zero_grad()
             # Forward
            target = model(img0,img1)  
            target = target.squeeze(1)
            loss = criterion(target, label.float())
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        val_running_loss = 0.0
        correct = 0.0
#         right, error = 0, 0
        #check validation loss after every epoch
        with torch.no_grad():
            model.eval()
            for i, data in enumerate(val_loader,0):
                img1, img2, label = data
                img1 = img1.to(device)
                img2 = img2.to(device)
                label = label.to(device)       
                target = model(img1,img2)  
                output = target.data.cpu().numpy()
#                 pred = np.argmax(output)
#                 if pred == label: 
#                     right += 1
#                     correct += 1
#                 else: 
#                     error += 1
                
                target = target.squeeze(1)
                loss = criterion(target, label.float())
                val_running_loss += loss.item()
#         print(right, error, right + error)
#         accuracy.append(right*1.0/(right+error))    
        avg_val_loss = val_running_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        accuracy_for_epoch = correct/len(val_loader)
#         accuracy.append(accuracy_for_epoch)
       
        print('Epoch [{}/{}],Train Loss: {:.4f}, Valid Loss: {:.8f}, Accuracy: {:.8f}'
            .format(epoch+1, num_epochs, avg_train_loss, avg_val_loss, accuracy_for_epoch))
    print("Finished Training")  
    return train_losses, val_losses, accuracy


In [ ]:
class ShopeeImageNet(nn.Module):
    def __init__(self):
        super(ShopeeImageNet, self).__init__()
        
        # Conv2d(input_channels, output_channels, kernel_size)
        self.conv1 = nn.Conv2d(3, 64, 3) 
        self.conv2 = nn.Conv2d(64, 128, 3)  
        self.conv3 = nn.Conv2d(128, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 3)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 21 * 21, 200)
        self.fcOut = nn.Linear(200, 1)
        self.sigmoid = nn.Sigmoid()
    
    def convs(self, x): 
        #32, 3, 200, 200
#         print(1, x.shape)
        x = F.relu(self.bn1(self.conv1(x)))
        # 32, 64, 198, 198
#         print(2, x.shape)
        x = F.max_pool2d(x, (2,2))
#         print(3, x.shape)
        # 32, 64, 99, 99
        x = F.relu(self.bn2(self.conv2(x)))
#         print(4, x.shape)
        # 32, 128, 97, 97
        x = F.max_pool2d(x, (2,2))
#         print(5, x.shape)
        # 32, 128, 48, 48
        x = F.relu(self.bn3(self.conv3(x)))
#         print(6, x.shape)
        # 32, 128, 46, 46
        x = F.max_pool2d(x, (2,2))
#         print(7, x.shape)
        # 32, 128, 23, 23
        x = F.relu(self.bn4(self.conv4(x)))
#         print(8, x.shape)
        # 32, 256, 21, 21
        return x

    def forward(self, x1, x2):        
        x1 = self.convs(x1)
#         print('forward')
#         print(9, x1.shape)
        x1 = x1.view(-1, 256 * 21 * 21)
#         print(10, x1.shape)
        x1 = self.sigmoid(self.fc1(x1))
#         print(11, x1.shape)
        x2 = self.convs(x2)
#         print(12, x2.shape)
        x2 = x2.view(-1, 256 * 21 * 21)
#         print(13, x2.shape)
        x2 = self.sigmoid(self.fc1(x2))
#         print(14, x2.shape)
        x = torch.abs(x1 - x2)
        x = self.fcOut(x)
#         print(15, x.shape)
        return x

In [ ]:
def imshow(img,text=None,should_save=False):
#     fig = plt.figure()
    plt.rcParams["figure.figsize"] = (40,20)
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [ ]:



dataiter = iter(validation_loader)


example_batch = next(dataiter)
concatenated = torch.cat((example_batch[0],example_batch[1]),0)
imshow(torchvision.utils.make_grid(concatenated))
print(example_batch[2].numpy())


In [ ]:
def train_only(model, train_loader, num_epochs):
    train_losses = []
    val_losses = []
    accuracy = []
    cur_step = 0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        print("Starting epoch " + str(epoch+1))
        for i, data in enumerate(train_loader,0):
            img0, img1, label = data
            img0, img1, label = img0.to(device), img1.to(device) , label.to(device)
            optimizer.zero_grad()
             # Forward
            target = model(img0,img1)  
            target = target.squeeze(1)
            loss = criterion(target, label.float())
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        print('Epoch [{}/{}],Train Loss: {:.4f}'
            .format(epoch+1, num_epochs, avg_train_loss))
    print("Finished Training")  
    return model, train_losses

In [ ]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Train the model
net = ShopeeImageNet().to(device)
model, train_loss = train_only(net, train_loader, 20)
torch.save(model.state_dict(), 'model_20_epochs.pth')
print("Model Saved Successfully")

In [ ]:
def predict(model, val_loader):
    train_losses = []
    val_losses = []
    accuracy = []
    cur_step = 0
    val_running_loss = 0.0
    correct = 0.0
    right, error = 0, 0
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(val_loader,0):
            img1, img2, label = data
            img1 = img1.to(device)
            img2 = img2.to(device)
            label = label.to(device)       
            target = model(img1,img2)              
            target = target.squeeze(1)
            output = target.data.cpu().numpy()
            
            pred = np.argmax(output)
            if pred == 0: 
                right += 1
                correct += 1
            else: 
                error += 1
            print(label, target, pred)
            loss = criterion(target, label.float())
            val_running_loss += loss.item()
        print(right, error, right + error)
        accuracy.append(right*1.0/(right+error))    
        avg_val_loss = val_running_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        accuracy_for_epoch = correct/len(val_loader)
        accuracy.append(accuracy_for_epoch)

In [ ]:
predict(model, validation_loader)

In [ ]:
#transfer learning
from torchvision import *
net = models.resnet18(pretrained=True)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 128)
net.fc = net.fc.cuda() 

In [ ]:
train_dataloader, validation_dataloader = create_data_sets(1000, 32)

In [ ]:
n_epochs = 5
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_dataloader)
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for i, data in enumerate(train_dataloader,0):
        img0, img1, label = data
        img0, img1, label = img0.to(device), img1.to(device) , label.to(device)
#     for batch_idx, (data_, target_) in enumerate(train_dataloader):
#         data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        
        outputs = net(img0, img1)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        net.eval()
#         for data_t, target_t in (test_dataloader):
#             data_t, target_t = data_t.to(device), target_t.to(device)
        for i, data in enumerate(validation_dataloader,0):
            img0, img1, label = data
            img0, img1, label = img0.to(device), img1.to(device) , label.to(device)
            outputs_t = net(img0, img1)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(validation_dataloader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')

        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    net.train()

In [ ]:
class Img2VecResnet18():
    def __init__(self):
        self.device = torch.device("cpu")
        self.numberFeatures = 512
        self.modelName = "resnet-18"
        self.model, self.featureLayer = self.getFeatureLayer()
        self.model = self.model.to(self.device)
        self.model.eval()
        self.toTensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
def getFeatureLayer(self):
    cnnModel = models.resnet18(pretrained=True)
    layer = cnnModel._modules.get('avgpool')
    self.layer_output_size = 512
    
    return cnnModel, layer

def getVec(self, img):
    image = self.normalize(self.toTensor(img)).unsqueeze(0).to(self.device)
    embedding = torch.zeros(1, self.numberFeatures, 1, 1)
    def copyData(m, i, o): embedding.copy_(o.data)
    h = self.featureLayer.register_forward_hook(copyData)
    self.model(image)
    h.remove()
    return embedding.numpy()[0, :, 0, 0]

In [ ]:
def getSimilarityMatrix(vectors):
    v = np.array(list(vectors.values())).T
    sim = np.inner(v.T, v.T) / ((np.linalg.norm(v, axis=0).reshape(-1,1)) * ((np.linalg.norm(v, axis=0).reshape(-1,1)).T))
    keys = list(vectors.keys())
    matrix = pd.DataFrame(sim, columns = keys, index = keys)
    return matrix

img2vec = Img2VecResnet18()
allVectors = {}
for image in tqdm(os.listdir("inputImagesCNN")):
    I = Image.open(os.path.join("inputImagesCNN", image))
    vec = img2vec.getVec(I)
    allVectors[image] = vec
    I.close()
similarityMatrix = getSimilarityMatrix(allVectors)

In [ ]:
class SiameseNetwork1(nn.Module):
    def __init__(self):
        super(SiameseNetwork1, self).__init__()
        # Conv2d(input_channels, output_channels, kernel_size)
        self.conv1 = nn.Conv2d(3, 64, 3) 
        self.conv2 = nn.Conv2d(64, 128, 3)  
        self.conv3 = nn.Conv2d(128, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 3)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 21 * 21, 200)
        self.fcOut = nn.Linear(200, 1)
        self.sigmoid = nn.Sigmoid()
    
    def convs(self, x): 
        #32, 3, 200, 200
#         print(1, x.shape)
        x = F.relu(self.bn1(self.conv1(x)))
        # 32, 64, 198, 198
#         print(2, x.shape)
        x = F.max_pool2d(x, (2,2))
#         print(3, x.shape)
        # 32, 64, 99, 99
        x = F.relu(self.bn2(self.conv2(x)))
#         print(4, x.shape)
        # 32, 128, 97, 97
        x = F.max_pool2d(x, (2,2))
#         print(5, x.shape)
        # 32, 128, 48, 48
        x = F.relu(self.bn3(self.conv3(x)))
#         print(6, x.shape)
        # 32, 128, 46, 46
        x = F.max_pool2d(x, (2,2))
#         print(7, x.shape)
        # 32, 128, 23, 23
        x = F.relu(self.bn4(self.conv4(x)))
#         print(8, x.shape)
        # 32, 256, 21, 21
        return x

    def forward(self, x1, x2):        
        x1 = self.convs(x1)
#         print('forward')
#         print(9, x1.shape)
        x1 = x1.view(-1, 256 * 21 * 21)
#         print(10, x1.shape)
        x1 = self.fc1(x1)
#         print(11, x1.shape)
        x2 = self.convs(x2)
#         print(12, x2.shape)
        x2 = x2.view(-1, 256 * 21 * 21)
#         print(13, x2.shape)
        x2 = self.fc1(x2)   
#         print(14, x2.shape)
        return x1, x2

#     def forward_once(self, x):
#         output = self.convs(x)
#         output = output.view(output.size()[0], -1)
#         output = self.fc1(output)
#         return output

#     def forward(self, input1, input2):
#         output1 = self.forward_once(input1)
#         output2 = self.forward_once(input2)
#         return output1, output2


class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [ ]:
def train_only(model, train_loader, num_epochs):
    train_losses = []
    val_losses = []
    accuracy = []
    cur_step = 0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        print("Starting epoch " + str(epoch+1))
        for i, data in enumerate(train_loader,0):
            img0, img1, label = data
            img0, img1, label = img0.to(device), img1.to(device) , label.to(device)
            optimizer.zero_grad()
             # Forward
            output1,output2 = model(img0,img1)
            loss_contrastive = criterion(output1,output2,label)
            loss_contrastive.backward()
            optimizer.step()
            running_loss += loss_contrastive.item()
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        print('Epoch [{}/{}],Train Loss: {:.4f}'
            .format(epoch+1, num_epochs, avg_train_loss))
    print("Finished Training")  
    return model, train_losses


In [ ]:
#Train with 64 batch size at a time

train_loader, validation_loader = create_data_sets(5000, batch_size = 64) #randomized samples
model2 = SiameseNetwork1().cuda()
criterion = ContrastiveLoss()
optimizer = optim.Adam(model2.parameters(),lr = 0.0005)

In [ ]:
model_out, train_loss = train_only(model2, train_loader, num_epochs=10)

fig = plt.figure()
plt.plot(train_loss, label ='Train Loss')
plt.legend()
plt.show();

In [ ]:
torch.save(model_out.state_dict(), 'model__7000_128__6_1_11_45.pth')
print("Model Saved Successfully")

In [ ]:
model = SiameseNetwork1().cuda()
model.load_state_dict(torch.load('../input/shopee/model__7000_128__6_1_11_45.pth'))
model.eval()


In [ ]:
train_loader, validation_loader = create_data_sets(5000, batch_size = 8) #randomized samples
count = 80
correct = 0
for i in range(0,10):
    dataiter = iter(validation_loader)

    example_batch = next(dataiter)
    concatenated = torch.cat((example_batch[0],example_batch[1]),0)
    labels = example_batch[2]
    
    print(labels)
    img0, img1 , label = example_batch[0].cuda(), example_batch[1].cuda() , example_batch[2].cuda()

    output1,output2 = model(img0,img1)
    euclidean_distance = F.pairwise_distance(output1, output2)
    print(euclidean_distance)
    target = torch.randn(1, 8, dtype=torch.double).to(device)
    target.fill_(1)
    output_labels = (euclidean_distance < target).long()
    print(output_labels)
#     correct = (labels == output_labels).long().sum()
#     print(correct)
    correct += torch.sum(label == output_labels).item()
    print(f"correct = {correct}")
    imshow(torchvision.utils.make_grid(concatenated))
#     correct_t += torch.sum(pred_t==target_t).item()
#     total_t += target_t.size(0)
    
accuracy = correct/count
print(f"Accuracy={accuracy}")